In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import sys
import pandas as pd
sys.path.append(r"C:\Users\minkyu\Desktop\K-TACC-main\utils") # 깃헙에서 레포 다운 받은 path 지정 해줘야함 
from BertAugmentation import BertAugmentation


In [ ]:
BERT_aug = BertAugmentation()
random_masking_insertion = BERT_aug.random_masking_insertion

In [ ]:
df = pd.read_csv('train_df.csv')

In [ ]:
filtered_df = df[df['공종(중분류)'] != '철근콘크리트공사']

In [ ]:
from tqdm import tqdm
tqdm.pandas()

filtered_df['aug'] = filtered_df['사고원인'].progress_apply(
    lambda sentence: random_masking_insertion(sentence, 0.2) if isinstance(sentence, str) else sentence
)


In [ ]:
filtered_df

In [ ]:
BERT_aug = BertAugmentation()
random_masking_replacement = BERT_aug.random_masking_replacement
random_masking_insertion = BERT_aug.random_masking_insertion

In [ ]:
import pandas as pd
from tqdm import tqdm

# filtered_df는 원본 DataFrame이라고 가정합니다.
augmented_df_rmr = filtered_df.copy()
augmented_df_rmi = filtered_df.copy()

ratio = 0.2  # 증강 비율

for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df), desc="증강 진행"):
    sentence = row['사고원인']
    
    # 문자열이 아닐 경우 문자열로 변환 (예: NaN 또는 float인 경우)
    if not isinstance(sentence, str):
        sentence = str(sentence)
    
    # 두 가지 증강 방법 적용
    augmented_sentence_rmr = random_masking_replacement(sentence, ratio)
    augmented_sentence_rmi = random_masking_insertion(sentence, ratio)
    
    # 증강된 결과를 각 DataFrame에 대체
    augmented_df_rmr.at[idx, '사고원인'] = augmented_sentence_rmr
    augmented_df_rmi.at[idx, '사고원인'] = augmented_sentence_rmi

In [ ]:
augmented_df_rmr['사고원인'].head()

In [ ]:
augmented_df_rmi['사고원인'].head()

In [ ]:
filtered_df = filtered_df.reset_index(drop=True)
augmented_df_rmr = augmented_df_rmr.reset_index(drop=True)
augmented_df_rmi = augmented_df_rmi.reset_index(drop=True)

In [ ]:
augmented_df_rmr = augmented_df_rmr.drop(columns= ['aug','cosine_similarity','bert_similarity'])
augmented_df_rmi = augmented_df_rmi.drop(columns= ['aug','cosine_similarity','bert_similarity'])

In [ ]:
augmented_df_rmr.to_csv("augmented_df_rmr.csv", index=False, encoding='utf-8-sig')
augmented_df_rmi.to_csv("augmented_df_rmi.csv", index=False, encoding='utf-8-sig')


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('train_df.csv')
augmented_df_rmr = pd.read_csv("augmented_df_rmr.csv")

In [ ]:
len(train)

In [ ]:
len(augmented_df_rmr)

In [ ]:
combined_df = pd.concat([train, augmented_df_rmr], ignore_index=True)


In [ ]:
combined_df2 = pd.concat([combined_df, augmented_df_rmr], ignore_index=True)


In [ ]:
combined_df.to_csv("train_rmi.csv", index=False, encoding='utf-8-sig')
combined_df2.to_csv("train_rmi_rmr.csv", index=False, encoding='utf-8-sig')

In [ ]:
combined_df.to_csv("train+rmr.csv", index=False, encoding='utf-8-sig')